In [1]:
!pip install pytorch-metric-learning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 10.1 MB/s eta 0:00:00


In [2]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.2 MB/s eta 0:00:00


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cifar10-imb-5cv/imb_train_data/1_trn_img_imb
/kaggle/input/cifar10-imb-5cv/imb_train_data/4_trn_img_imb
/kaggle/input/cifar10-imb-5cv/imb_train_data/2_trn_img_imb
/kaggle/input/cifar10-imb-5cv/imb_train_data/0_trn_img_imb
/kaggle/input/cifar10-imb-5cv/imb_train_data/3_trn_img_imb
/kaggle/input/cifar10-imb-5cv/imb_train_label/3_trn_lab_imb
/kaggle/input/cifar10-imb-5cv/imb_train_label/0_trn_lab_imb
/kaggle/input/cifar10-imb-5cv/imb_train_label/4_trn_lab_imb
/kaggle/input/cifar10-imb-5cv/imb_train_label/2_trn_lab_imb
/kaggle/input/cifar10-imb-5cv/imb_train_label/1_trn_lab_imb


In [4]:
import collections
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from sklearn.neighbors import NearestNeighbors
import time
import torchvision
import torch.nn.functional as F
import math
import matplotlib.pyplot as plt
from torch import optim
from sklearn.metrics import classification_report

from pytorch_metric_learning import miners, losses, distances, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

print(torch.version.cuda)

11.0


In [5]:

t3 = time.time()

##############################################################################
"""args for AE"""

args = {}
args['dim_h'] = 64         # factor controlling size of hidden layers
args['n_channel'] = 3#1    # number of channels in the input data 

args['n_z'] = 600 #300     # number of dimensions in latent space. 

args['sigma'] = 1.0        # variance in n_z
args['lambda'] = 0.01      # hyper param for weight of discriminator loss
args['lr'] = 0.0002        # learning rate for Adam optimizer .000
args['epochs'] = 1000       # how many epochs to run for
args['batch_size'] = 100   # batch size for SGD
args['save'] = True        # save weights at each epoch of training if True
args['train'] = True       # train networks if True, else load networks from
args['triplet_margin'] = 0.4 # default 0.2, use 0.4 to try

args['dataset'] = 'cifar10'#'mnist'  #'fmnist' # specify which dataset to use


##############################################################################



## create encoder model and decoder model
class Encoder(nn.Module):
    def __init__(self, args):
        super(Encoder, self).__init__()

        self.n_channel = args['n_channel']
        self.dim_h = args['dim_h']
        self.n_z = args['n_z']
        
        # convolutional filters, work excellent with image data
        self.conv = nn.Sequential(
            nn.Conv2d(self.n_channel, self.dim_h, 4, 2, 1, bias=False),
            #nn.ReLU(True),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(self.dim_h, self.dim_h * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.dim_h * 2),
            #nn.ReLU(True),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(self.dim_h * 2, self.dim_h * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.dim_h * 4),
            #nn.ReLU(True),
            nn.LeakyReLU(0.2, inplace=True),
            
            
            #nn.Conv2d(self.dim_h * 4, self.dim_h * 8, 4, 2, 1, bias=False),
            
            
            #3d and 32 by 32
            nn.Conv2d(self.dim_h * 4, self.dim_h * 8, 4, 1, 0, bias=False),
            
            nn.BatchNorm2d(self.dim_h * 8), # 40 X 8 = 320
            #nn.ReLU(True))
            nn.LeakyReLU(0.2, inplace=True)) #,
            #nn.Conv2d(self.dim_h * 8, 1, 2, 1, 0, bias=False))
            #nn.Conv2d(self.dim_h * 8, 1, 4, 1, 0, bias=False))
        # final layer is fully connected
        self.fc = nn.Linear(self.dim_h * (2 ** 3), self.n_z)
        
        

    def forward(self, x):
        #print('enc')
        #print('input ',x.size()) #torch.Size([100, 3,32,32])
        x = self.conv(x)
        
        x = x.squeeze()
        #print('aft squeeze ',x.size()) #torch.Size([128, 320])
        #aft squeeze  torch.Size([100, 320])
        x = self.fc(x)
        #print('out ',x.size()) #torch.Size([128, 20])
        #out  torch.Size([100, 300])
        return x


class Decoder(nn.Module):
    def __init__(self, args):
        super(Decoder, self).__init__()

        self.n_channel = args['n_channel']
        self.dim_h = args['dim_h']
        self.n_z = args['n_z']

        # first layer is fully connected
        self.fc = nn.Sequential(
            nn.Linear(self.n_z, self.dim_h * 8 * 9 * 9),
            nn.ReLU())

        # deconvolutional filters, essentially inverse of convolutional filters
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(self.dim_h * 8, self.dim_h * 4, 4),
            nn.BatchNorm2d(self.dim_h * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(self.dim_h * 4, self.dim_h * 2, 4),
            nn.BatchNorm2d(self.dim_h * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(self.dim_h * 2, 3, 4, stride=2),
            #nn.Sigmoid())
            nn.Tanh())

    def forward(self, x):
        #print('dec')
        #print('input ',x.size())
        x = self.fc(x)
        x = x.view(-1, self.dim_h * 8, 9, 9)
        x = self.deconv(x)
        return x

##############################################################################
"""set models, loss functions"""
# control which parameters are frozen / free for optimization
def free_params(module: nn.Module):
    for p in module.parameters():
        p.requires_grad = True

def frozen_params(module: nn.Module):
    for p in module.parameters():
        p.requires_grad = False


##############################################################################
"""functions to create SMOTE images"""

def biased_get_class(c):
    
    xbeg = dec_x[dec_y == c]
    ybeg = dec_y[dec_y == c]
    
    return xbeg, ybeg
    #return xclass, yclass


def G_SM(X, y,n_to_sample,cl):

    # determining the number of samples to generate
    #n_to_sample = 10 

    # fitting the model
    n_neigh = 5 + 1
    nn = NearestNeighbors(n_neighbors=n_neigh, n_jobs=1)
    nn.fit(X)
    dist, ind = nn.kneighbors(X)

    # generating samples
    base_indices = np.random.choice(list(range(len(X))),n_to_sample)
    neighbor_indices = np.random.choice(list(range(1, n_neigh)),n_to_sample)

    X_base = X[base_indices]
    X_neighbor = X[ind[base_indices, neighbor_indices]]

    samples = X_base + np.multiply(np.random.rand(n_to_sample,1),
            X_neighbor - X_base)

    #use 10 as label because 0 to 9 real classes and 1 fake/smoted = 10
    return samples, [cl]*n_to_sample

#xsamp, ysamp = SM(xclass,yclass)

###############################################################################


#NOTE: Download the training ('.../0_trn_img.txt') and label files 
# ('.../0_trn_lab.txt').  Place the files in directories (e.g., ../MNIST/trn_img/
# and /MNIST/trn_lab/).  Originally, when the code was written, it was for 5 fold
#cross validation and hence there were 5 files in each of the 
#directories.  Here, for illustration, we use only 1 training and 1 label
#file (e.g., '.../0_trn_img.txt' and '.../0_trn_lab.txt').

dtrnimg = '/kaggle/input/cifar10-imb-5cv/imb_train_data'
dtrnlab = '/kaggle/input/cifar10-imb-5cv/imb_train_label'

ids = os.listdir(dtrnimg)
idtri_f = [os.path.join(dtrnimg, image_id) for image_id in ids]
idtri_f.sort()
print(idtri_f)

ids = os.listdir(dtrnlab)
idtrl_f = [os.path.join(dtrnlab, image_id) for image_id in ids]
idtrl_f.sort()
print(idtrl_f)

#for i in range(5):
for k in range(1):
    print()
    print(k)
    encoder = Encoder(args)
    decoder = Decoder(args)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(device)
    decoder = decoder.to(device)
    encoder = encoder.to(device)

    train_on_gpu = torch.cuda.is_available()

    #decoder loss function
    criterion = nn.MSELoss()
    criterion = criterion.to(device)
    
    trnimgfile = idtri_f[k]
    trnlabfile = idtrl_f[k]
    
    print(trnimgfile)
    print(trnlabfile)
    
    dec_x = np.loadtxt(trnimgfile)
    dec_y = np.loadtxt(trnlabfile)

    print('train imgs before reshape ',dec_x.shape) 
    print('train labels ',dec_y.shape) 
    print(collections.Counter(dec_y))
    dec_x = dec_x.reshape(dec_x.shape[0],3,32,32)   
    print('train imgs after reshape ',dec_x.shape) 

    batch_size = 100
    num_workers = 0

    #torch.Tensor returns float so if want long then use torch.tensor
    tensor_x = torch.Tensor(dec_x)
    tensor_y = torch.tensor(dec_y,dtype=torch.long)
    cifar10_bal = TensorDataset(tensor_x,tensor_y) 
    train_loader = torch.utils.data.DataLoader(cifar10_bal, 
        batch_size=batch_size,shuffle=True,num_workers=num_workers)
    
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    

    best_loss = np.inf
    
    # metric leanring##############################################################################
    distance = distances.CosineSimilarity()
    reducer = reducers.ThresholdReducer(low=0)
    loss_fn = losses.TripletMarginLoss(margin=args['triplet_margin'], distance=distance, reducer=reducer)
    miner = miners.TripletMarginMiner(margin=args['triplet_margin'], distance=distance, type_of_triplets='semihard')
    accuracy_calculator = AccuracyCalculator(include=('precision_at_1',), k=1)
    ###############################################################################################

    t0 = time.time()
    if args['train']:
        enc_optim = torch.optim.Adam(encoder.parameters(), lr = args['lr'])
        dec_optim = torch.optim.Adam(decoder.parameters(), lr = args['lr'])
    
        for epoch in range(args['epochs']):
            train_loss = 0.0
            tmse_loss = 0.0
            tdiscr_loss = 0.0
            contrastive_loss = 0.0
            # train for one epoch -- set nets to train mode
            encoder.train()
            decoder.train()
        
            for i, (images,labs) in enumerate(train_loader):
            
                # zero gradients for each batch
                encoder.zero_grad()
                decoder.zero_grad()
                #print(images)
                images, labs = images.to(device), labs.to(device)
                #print('images ',images.size()) 
                labsn = labs.detach().cpu().numpy()
                #print('labsn ',labsn.shape, labsn)
            
                # run images
                z_hat = encoder(images)
            
                x_hat = decoder(z_hat) #decoder outputs tanh
                #print('xhat ', x_hat.size())
                #print(x_hat)
                mse = criterion(x_hat,images)
                #print('mse ',mse)
                
                       
                resx = []
                resy = []
            
                tc = np.random.choice(10,1)
                #tc = 9
                xbeg = dec_x[dec_y == tc]
                ybeg = dec_y[dec_y == tc] 
                xlen = len(xbeg)
                nsamp = min(xlen, 100)
                ind = np.random.choice(list(range(len(xbeg))),nsamp,replace=False)
                xclass = xbeg[ind]
                yclass = ybeg[ind]
            
                xclen = len(xclass)
                #print('xclen ',xclen)
                xcminus = np.arange(1,xclen)
                #print('minus ',xcminus.shape,xcminus)
                
                xcplus = np.append(xcminus,0)
                #print('xcplus ',xcplus)
                xcnew = (xclass[[xcplus],:])
                #xcnew = np.squeeze(xcnew)
                xcnew = xcnew.reshape(xcnew.shape[1],xcnew.shape[2],xcnew.shape[3],xcnew.shape[4])
                #print('xcnew ',xcnew.shape)
            
                xcnew = torch.Tensor(xcnew)
                xcnew = xcnew.to(device)
            
                #encode xclass to feature space
                xclass = torch.Tensor(xclass)
                xclass = xclass.to(device)
                xclass = encoder(xclass)
                #print('xclass ',xclass.shape) 
            
                xclass = xclass.detach().cpu().numpy()
            
                xc_enc = (xclass[[xcplus],:])
                xc_enc = np.squeeze(xc_enc)
                #print('xc enc ',xc_enc.shape)
            
                xc_enc = torch.Tensor(xc_enc)
                xc_enc = xc_enc.to(device)
                
                ximg = decoder(xc_enc)
                
                mse2 = criterion(ximg,xcnew)
                
                # Start contrastive loss term
                embeddings = encoder(images)
                indices_tuple = miner(embeddings, labs)
                contra_loss = loss_fn(embeddings, labs, indices_tuple)
                '''
                if i % 100 ==0:
                    print(
                    "Epoch {} Iteration {}: Loss = {}, Number of mined triplets = {}".format(
                    epoch, i, contra_loss, miner.num_triplets)
                    )'''
            
                comb_loss = mse2 + mse + contra_loss
                comb_loss.backward()
            
                enc_optim.step()
                dec_optim.step()
            
                train_loss += comb_loss.item()*images.size(0)
                tmse_loss += mse.item()*images.size(0)
                tdiscr_loss += mse2.item()*images.size(0)
                contrastive_loss += contra_loss.item() * images.size(0)
                 
            # print avg training statistics 
            train_loss = train_loss/len(train_loader)
            tmse_loss = tmse_loss/len(train_loader)
            tdiscr_loss = tdiscr_loss/len(train_loader)
            contrastive_loss = contrastive_loss / len(train_loader)
            print('Epoch: {} \tTrain Loss: {:.6f} \tmse loss: {:.6f} \tmse2 loss: {:.6f} \tcontrastive loss: {:.6f}'.format(epoch,
                    train_loss,tmse_loss,tdiscr_loss, contrastive_loss))
            
        
        
            #store the best encoder and decoder models
            #here, /crs5 is a reference to 5 way cross validation, but is not
            #necessary for illustration purposes
            if train_loss < best_loss:
                print('Saving..')
                #path_enc = './models/crs5/' \
                    #+ str(i) + '/bst_enc.pth'
                path_enc = './'+ str(k) +'_bst_enc.pth'
                #path_dec = './models/crs5/' \
                    #+ str(i) + '/bst_dec.pth'
                path_dec = './'+ str(k) +'_bst_dec.pth'
             
                torch.save(encoder.state_dict(), path_enc)
                torch.save(decoder.state_dict(), path_dec)
        
                best_loss = train_loss
        
        
        #in addition, store the final model (may not be the best) for
        #informational purposes
        path_enc = './' + str(k) + '_final_enc.pth'
        path_dec = './' + str(k) + '_final_dec.pth'
        print(path_enc)
        print(path_dec)
        #torch.save(encoder.state_dict(), path_enc)
        #torch.save(decoder.state_dict(), path_dec)
        print()
              
    t1 = time.time()
    print('total time(min): {:.2f}'.format((t1 - t0)/60))             

    
t4 = time.time()
print('final time(min): {:.2f}'.format((t4 - t3)/60))

['/kaggle/input/cifar10-imb-5cv/imb_train_data/0_trn_img_imb', '/kaggle/input/cifar10-imb-5cv/imb_train_data/1_trn_img_imb', '/kaggle/input/cifar10-imb-5cv/imb_train_data/2_trn_img_imb', '/kaggle/input/cifar10-imb-5cv/imb_train_data/3_trn_img_imb', '/kaggle/input/cifar10-imb-5cv/imb_train_data/4_trn_img_imb']
['/kaggle/input/cifar10-imb-5cv/imb_train_label/0_trn_lab_imb', '/kaggle/input/cifar10-imb-5cv/imb_train_label/1_trn_lab_imb', '/kaggle/input/cifar10-imb-5cv/imb_train_label/2_trn_lab_imb', '/kaggle/input/cifar10-imb-5cv/imb_train_label/3_trn_lab_imb', '/kaggle/input/cifar10-imb-5cv/imb_train_label/4_trn_lab_imb']

0
cuda
/kaggle/input/cifar10-imb-5cv/imb_train_data/0_trn_img_imb
/kaggle/input/cifar10-imb-5cv/imb_train_label/0_trn_lab_imb
train imgs before reshape  (10280, 3072)
train labels  (10280,)
Counter({0.0: 4500, 1.0: 2000, 2.0: 1000, 3.0: 800, 4.0: 600, 5.0: 500, 6.0: 400, 7.0: 250, 8.0: 150, 9.0: 80})
train imgs after reshape  (10280, 3, 32, 32)
Epoch: 0 	Train Loss: 57.